In [1]:
import requests
import pandas as pd
import numpy as np
from pybtex.database import BibliographyData, Entry
import re
# from utils.helper import search_studies, assert_list, get_citation, fetch_data
from utils.helper import *

class NOAAStudies:
    def __init__(self):
        """
        Initialize the NOAAStudies class with base URL and dictionaries to hold studies and data table indices.


        The search parameters are designed to be flexible and can vary for different searches using the same object.
        Instead of overwriting the initialized default parameters, each search maintains its own set of parameters, 
        allowing for independent configurations across multiple searches.
        """
        self.BASE_URL = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json"
        self.studies = {}
        self.data_table_index = {}

    def search_studies(self, xml_id=None, noaa_id=None, data_publisher="NOAA", data_type_id = None, keywords = None, investigators=None, 
                       max_lat=None, min_lat=None, max_lon=None, min_lon=None, location = None, publication = None, search_text = None, 
                       earliest_year = None, latest_year = None, cv_whats = None, recent = False):
        """
        Search for NOAA studies using specific search parameters.
        
        Parameters:
            xml_id (str): XML ID of the study. (Primary)
            noaa_id (str): NOAA study ID. (Primary)
            data_publisher (str): Data publisher's name. Default: NOAA 
            data_type_id (str): Data Type specific Studies.
            keywords (str): Keywords to search for.  
            investigators (str): Name(s) of investigators.
            min_lat/max_lat (float): Latitude range for location-based search.
            min_lat/max_lat (float): Longitude range for location-based search.
            location (str): Location description.
            species (str): FOUR letter code species code
            publication (str): Specific publication.
            search_text (str): Publication / studyNotes / String based search.
           
        """
        if noaa_id:
            params = {'NOAAStudyId': noaa_id}
        elif xml_id:
            params = {'xmlId': xml_id}
        else:
            params = {
            'dataPublisher' : data_publisher,
            'dataTypeId' : data_type_id,
            'keywords' : keywords, 
            'investigators' : investigators,
            'minLat' : min_lat,
            'maxLat' : max_lat,
            'minLon' : min_lon,
            'maxLon' : max_lon,
            'locations' : location,
            'searchText' : publication, 
            'searchText' : search_text,
            'cvWhats': cv_whats,
            'earliestYear' : earliest_year,
            'latestYear': latest_year, 
            'recent' : recent,
        }
        # Filtering out None values
        # params = {k: v for k, v in params.items() if v}
        # response = requests.get(self.BASE_URL, params=params)
        # if response.status_code == 200:
        #     self.response_parser(response.json())
        # else:
        #     print(f"Error fetching studies: {response.status_code}")

        self.response_parser(search_studies(params))

    def response_parser(self, data):
        """
        Parse the JSON response from NOAA and populate the studies dictionary.
        
        Parameters:
            data (dict): The JSON data returned from a search query.
        """
        for study in data.get('study', []):
            noaa_study_id = study.get('NOAAStudyId')
            xml_study_id = study.get('xmlId')
            self.studies[noaa_study_id] = {
                'base_meta': self.load_base_meta(study),
                'investigators': self.load_investigators(study),
                'publications': self.load_publications(study),
                'sites': self.load_sites(study, noaa_study_id),
                # 'number of sites': len(sites)
                'pageUrl' : study.get('onlineResourceLink', np.nan)
            }

    def load_base_meta(self, study):
        """
        Load base metadata for a study.
        
        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
        """
        fields = ['NOAAStudyId', 'studyName', 'dataType', 'earliestYearBP', 'mostRecentYearBP',
                  'earliestYearCE', 'mostRecentYearCE', 'studyNotes', 'scienceKeywords']
        return {field: study.get(field, np.nan) for field in fields}

    def load_investigators(self, study):
        """
        Extract investigator details from the study data.
        
        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
        """
        investigators = study.get("investigatorDetails", [])
        if investigators:
            return ", ".join([f"{i.get('firstName', 'N/A')} {i.get('lastName', 'N/A')}" for i in investigators])
        return np.nan

    def load_publications(self, study):
        """
        Extract and format publication data from the study as a dictionary.

        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
            study_id (str): The unique identifier for the NOAA study.

        Returns:
            list: A list of dictionaries representing publication details.
        """
        publications = []
        for pub in study.get('publication', []):
            author_info = pub.get('author', {})
            identifier_info = pub.get('identifier', {})

            # Extract fields for the publication dictionary
            publication_data = {
                'NOAAStudyId': study.get('NOAAStudyId'),
                'author': author_info.get('name', 'Unknown Author'),
                'title': pub.get('title', 'Unknown Title'),
                'journal': pub.get('journal', 'Unknown Journal'),
                'year': str(pub.get('pubYear', 'Unknown Year')),
                'volume': pub.get('volume', np.nan),
                'number': pub.get('issue', np.nan),
                'pages': pub.get('pages', np.nan),
                'type': pub.get('type', np.nan),
                'doi': identifier_info.get('id', np.nan) if identifier_info else np.nan,
                'url': identifier_info.get('url', np.nan) if identifier_info else np.nan
            }

            # Add the publication dictionary to the list
            publications.append(publication_data)
        
        return publications

    def load_sites(self, study, study_id):
        """
        Load and format site data associated with the study.
        
        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
            study_id (str): The unique identifier of the study for reference.
        """        
        return {
            site.get('NOAASiteId', np.nan): {
                'siteName': site.get('siteName', np.nan),
                'locationName': site.get('locationName', np.nan),
                'lat': site.get('geo', {}).get('geometry', {}).get('coordinates', [None, None])[0],
                'lon': site.get('geo', {}).get('geometry', {}).get('coordinates', [None, None])[1],
                'paleoData': self.load_paleo_data(site.get('paleoData', []), study_id, site.get('NOAASiteId'))
            }
            for site in study.get('site', [])
        }

    def load_paleo_data(self, paleoData, study_id, site_id):
        """
        Extract and format paleo data associated with a site.
        
        Parameters:
            paleoData (list): List of paleo data from the site.
            study_id (str): The unique identifier of the study.
            site_id (str): The unique identifier of the site.
        """
        paleo_dict = {}
        for paleo in paleoData:
            # Safe access to 'dataFile' list
            data_files = paleo.get('dataFile', [])
            file_url = data_files[0].get('fileUrl', np.nan) if data_files else np.nan
            variables = []
            if data_files:  # Check if 'dataFile' is not empty
                variables = [var.get('cvShortName', np.nan) for var in data_files[0].get('variables', [])]
            data_table_id = paleo.get('NOAADataTableId', np.nan)
            paleo_details = {
                'NOAADataTableId': data_table_id,
                'dataTableName': paleo.get('dataTableName', np.nan),
                'timeUnit': paleo.get('timeUnit', np.nan),
                'fileUrl': file_url,
                'variables': variables
            }
            paleo_dict[data_table_id] = paleo_details

            self.data_table_index[data_table_id] = {
                'file_url': file_url,
                'study_id': study_id,
                'site_id': site_id
            }
        return paleo_dict

        
    def get_response(self):
        """
        Compile and return a DataFrame of all loaded studies along with their detailed metadata and linked information.
        
        Returns:
            DataFrame: A DataFrame representing the consolidated data of all studies, including metadata, investigators,
                       publications, and site details.
        """
        data = [{
            **study['base_meta'],
            'Investigators': study['investigators'],
            'publications': study['publications'],
            'sites': study['sites']
        } for study in self.studies.values()]
        return pd.DataFrame(data)

    def get_publications(self, study_ids, output_format="dataframe"):
        """
        Return publications for one or more study IDs in the specified format.

        Parameters:
            study_ids (list or str): Single or list of NOAAStudyIds.
            output_format (str): The desired output format. Options:
                - "dataframe": Returns a pandas DataFrame of publication details.
                - "bibtex": Returns a pybtex.database.BibliographyData object.

        Returns:
            pd.DataFrame or pybtex.database.BibliographyData: Publications data in the chosen format.
        """
        study_ids = assert_list(study_ids)  # Ensure study_ids is a list
        
        if output_format == "dataframe":            
            dfs = []
            for study_id in study_ids:
                if study_id in self.studies:
                    publications = self.studies[study_id].get('publications', [])
                    df = pd.DataFrame(publications)
                    # df.set_index('NOAAStudyId', inplace=True)  # Set NOAAStudyId as the index
                    dfs.append(df)
                else:
                    print(f"Study ID {study_id} not found.")
            display(pd.concat(dfs))
            return pd.concat(dfs) if dfs else pd.DataFrame()

        elif output_format == "bibtex":            
            bib_entries = {}
            for study_id in study_ids:
                if study_id in self.studies:
                    publications = self.studies[study_id].get('publications', [])
                    for pub in publications:
                        fields = {k: v for k, v in pub.items() if k not in ['NOAAStudyId'] and v}
                        entry = Entry('article', fields=fields)
                        citation_key = get_citation(pub)
                        bib_entries[citation_key] = entry
                else:
                    print(f"Study ID {study_id} not found.")
            
            return BibliographyData(entries=bib_entries)

        else:
            print("Invalid output_format. Choose 'dataframe' or 'bibtex'.")

    def get_sites(self, study_ids):
        """
        Return a DataFrame of sites for one or more study IDs.

        Parameters:
            study_ids (list or str): Single or list of NOAAStudyIds.

        Returns:
            pd.DataFrame: Sites DataFrame.
        """
        study_ids = assert_list(study_ids)  # Convert to list if single ID
        
        dfs = []
        for study_id in study_ids:
            if study_id not in self.studies:
                print(f"Study ID {study_id} not found.")
                continue
            
            sites_data = self.studies[study_id].get('sites', {})
            sites_list = []
            for site_id, site_info in sites_data.items():
                site_info['NOAASiteId'] = site_id
                paleo_list = site_info.pop('paleoData', {})
                for paleo_id, paleo_info in paleo_list.items():
                    record = {**site_info, **paleo_info, 'NOAADataTableId': paleo_id, 'NOAAStudyId': study_id}
                    sites_list.append(record)

            if sites_list:
                df = pd.DataFrame(sites_list)
                dfs.append(df)
                # print(dfs)
        
        result = pd.concat(dfs) if dfs else pd.DataFrame()
        result.set_index('NOAAStudyId', inplace=True)  # Set NOAAStudyId as the index
        return result

    def get_data(self, dataTableIDs=None, file_urls=None):
        """
        Fetch and return the data for one or more dataTableIDs or file URLs.

        Parameters:
            dataTableIDs (list or str): Single or list of NOAADataTableIds.
            file_urls (list or str): Single or list of file URLs.

        Returns:
            pd.DataFrame: Combined DataFrame of all fetched data.
        """

        """ 
        @TODO: 
            Add attributes to each data frame
             
        """
        if dataTableIDs:
            dataTableIDs = assert_list(dataTableIDs)
            dfs = []
            for dataTableID in dataTableIDs:
                file_url = self.data_table_index.get(dataTableID, {}).get('file_url')
                if not file_url:
                    print(f"Data Table ID {dataTableID} not found or no associated file URL.")
                    continue
                print(f"File URL {type(file_url)}")
                df = fetch_data(file_url)

            study_id = self.data_table_index[dataTableID].get('study_id')
            site_id = self.data_table_index[dataTableID].get('site_id')
            study_data = self.studies.get(study_id, {})
            
            # Attach attributes to DataFrame
            df.attrs['NOAAStudyId'] = study_id
            df.attrs['Publication'] = study_data.get('publications', 'N/A')
            site_data = study_data.get('sites', {}).get(site_id, {})
            df.attrs['Lat'] = site_data.get('lat', 'N/A')
            df.attrs['Lon'] = site_data.get('lon', 'N/A')
            df.attrs['fileURL'] = file_url

            dfs.append(df)
            return dfs
        
        if file_urls:
            file_urls = assert_list(file_urls)  # Convert to list if single URL
            dfs = [fetch_data(file_url) for file_url in file_urls]
            return dfs

        print("No dataTableID or file URL provided.")
        return pd.DataFrame()

In [2]:
response = {"study": [
    {
      "xmlId": "16017",
      "NOAAStudyId": "18315",
      "studyName": "Makassar Strait - Single specimens of P. obliquiloculata d18O and d13C from 704-1851 AD",
      "dataPublisher": "NOAA",
      "dataType": "PALEOCEANOGRAPHY",
      "investigatorDetails": [
        {
          "firstName": "Deborah",
          "lastName": "Khider",
          "initials": "D.",
          "orcId": "0000-0001-7501-8430"
        },
        {
          "firstName": "Lowell",
          "lastName": "Stott",
          "initials": "L.D.",
          "orcId": "0000-0002-2025-0731"
        },
        {
          "firstName": "Julien",
          "lastName": "Emile-Geay",
          "initials": "J.",
          "orcId": "0000-0001-5920-4751"
        },
        {
          "firstName": "Robert",
          "lastName": "Thunell",
          "initials": "R.C.",
          "orcId": "0000-0001-7052-1707"
        },
        {
          "firstName": "Doug",
          "lastName": "Hammond",
          "initials": "D.E.",
          "orcId": None
        }
      ],
      "studyNotes": "This dataset contains the d18O, d13C, and weights of single specimens of P. obliquiloculata used in the reconstruction of ENSO variability over the past 2,000 years.",
      "onlineResourceLink": "https://www.ncei.noaa.gov/access/paleo-search/study/18315",
      "scienceKeywords": [
        "ENSO",
        "Medieval Climate Anomaly (MCA)",
        "Little Ice Age (LIA)"
      ],
      "earliestYearBP": 1246,
      "mostRecentYearBP": 99,
      "earliestYearCE": 704,
      "mostRecentYearCE": 1851,
      "publication": [
        {
          "author": {
            "name": "Khider, D"
          },
          "pubYear": 2011,
          "title": "Assessing El Nino Southern Oscillation variability during the past millennium",
          "journal": "Paleoceanography",
          "volume": "26",
          "edition": None,
          "issue": None,
          "pages": None,
          "reportNumber": "PA3222",
          "citation": "Khider, D., L. Stott, J. Emile-Geay, R. Thunell, and D.E. Hammond. 2011. Assessing El Nino Southern Oscillation variability during the past millennium. Paleoceanography, 26, PA3222. doi: 10.1029/2011PA002139",
          "type": "publication",
          "identifier": {
            "type": "doi",
            "id": "10.1029/2011PA002139",
            "url": "http://dx.doi.org/10.1029/2011PA002139"
          },
          "abstract": "We present a reconstruction of El Nino Southern Oscillation (ENSO) variability spanning the Medieval Climate Anomaly (MCA, A.D. 800-1300) and the Little Ice Age (LIA, A.D. 1500-1850). Changes in ENSO are estimated by comparing the spread and symmetry of d18O values of individual speciments of the thermocline-dwelling foraminifer Pulleniatina obliquiloculata extracted from discrete time horizons of a sediment core collected in the Sulawesi Sea, at the edge of the western tropical Pacific warm pool. The spread of individual d18O values is interpreted to be a measure of the strength of both phases of ENSO while the symmetry of the d18O distributions is used to evaluate the relative strength/frequency of El Nino and La Nina events. In contrast to previous studies, we use robust and resistant statistics to quantify the spread and symmetry of the d18O distributions; an approach motivated by the relatively small sample size and the presence of outliers. Furthermore, we use a pseudo-proxy to investigate the effects of the different paleo-environmental factors on the statistics of the d18O distributions, which could bias the paleo-ENSO reconstruction. We find no systematic difference in the magnitude/strength of ENSO during the Nothern Hemisphere MCA or LIA. However, our results suggest that ENSO during the MCA was kewed toward stronger/more frequent La Nina than El Nino, an observation consistent with the medieval megadroughts documented from sites in western North America.",
          "pubRank": "1"
        }
      ],
      "site": [
        {
          "NOAASiteId": "53040",
          "siteName": "MD98-2177",
          "locationName": "Ocean\u003EPacific Ocean\u003EWestern Pacific Ocean",
          "geo": {
            "geoType": "Feature",
            "geometry": {
              "type": "POINT",
              "coordinates": [
                "1.4033",
                "119.078"
              ]
            },
          },
          "paleoData": [
            {
              "dataTableName": "MD98-2177 isotopes Khider11",
              "NOAADataTableId": "28674",
              "timeUnit": "CE",
              "dataFile": [
                {
                  "fileUrl": "https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2011/khider2011.txt",
                  "variables": [
                    {
                      "cvShortName": None
                    },
                    {
                      "cvShortName": None
                    },
                    {
                      "cvShortName": None
                    },
                    {
                      "cvShortName": None
                    },
                    {
                      "cvShortName": None
                    },
                    {
                      "cvShortName": None
                    },
                    {
                      "cvShortName": None
                    }
                  ],
                }
              ]
            }
          ]
        }
      ],
    }
    ]}

In [3]:
noaaStudies = NOAAStudies() 

In [4]:
noaaStudies.response_parser(response)

In [5]:
res = noaaStudies.get_publications('18315')

,NOAAStudyId,author,title,journal,year,volume,number,pages,type,doi,url
0,18315,"Khider, D",Assessing El Nino Southern Oscillation variabi...,Paleoceanography,2011,26,None,None,publication,10.1029/2011PA002139,http://dx.doi.org/10.1029/2011PA002139


In [6]:
noaaStudies.get_data(dataTableIDs='28674')

File URL <class 'str'>
https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2011/khider2011.txt


[     depth_top depth_bottom age_ADbot age_ADtop d13CcarbVPDB d18OcarbVPDB  \
 0            0            1      1843      1851        0.936       -2.254   
 1            0            1      1843      1851        0.895       -2.356   
 2            0            1      1843      1851        0.514        -2.63   
 3            0            1      1843      1851          0.9        -2.48   
 4            0            1      1843      1851        0.957       -2.094   
 ...        ...          ...       ...       ...          ...          ...   
 1160        96           98       704       734        0.711       -1.719   
 1161        96           98       704       734        0.852        -1.69   
 1162        96           98       704       734         0.68       -2.093   
 1163        96           98       704       734         0.69       -2.023   
 1164        96           98       704       734        0.629       -2.361   
 
      wgt-ind\r  
 0         34\r  
 1         37\r  
 2      